In [1]:
from dcicutils import ff_utils
from functions.wfr import *
from functions.wfr_settings import *
from functions.notebook_functions import *

# get key from keypairs.json
my_env = 'data'
my_key = get_key('koray_data')

# url for hic exps
fastq_url = "search/?type=FileFastq&quality_metric.uuid=No+value" + \
            "&status=pre-release&status=released&status=released%20to%20project&status=uploaded" + \
            "&limit=all"

print(fastq_url)
fastq_files = ff_utils.search_metadata(fastq_url , key = my_key)
print(len(fastq_files))

search/?type=FileFastq&quality_metric.uuid=No+value&status=pre-release&status=released&status=released%20to%20project&status=uploaded&limit=all
0


In [7]:
# for a given experiment set and some parameters like instrument
# print set of files and their partA hic workflow status
# if there are one that are running report the number of running cases
# if there are file pairs that don't have a corresponding part A, report them separately

run_fastqc = input("Do you wanna run fastqc if missing? (fq/none)")



#if beyond run_time, assume failed
run_time = 24

counter = 0
completed = 0
print(len(fastq_files))

weird = 0
needs_run = 0
needs_run_again = 0
running = 0
for a_fastq in fastq_files:
    
    file_id = a_fastq['accession']  
    report = get_wfr_out(file_id, 'fastqc-0-11-4-1',  my_key, md_qc = True, run=run_time)

    if report['status'] == 'running':
        running += 1
        print(file_id, 'still running')
        continue
        
    attributions = get_attribution(a_fastq)
    
    # if we need a run
    if report['status'].startswith('no') or not report:
        needs_run += 1
        if run_fastqc == 'fq':
            print('fastqc running for', a_fastq['accession'])
            inp_f = {'input_fastq':file_id}
            run_missing_wfr(step_settings('fastqc-0-11-4-1', 'none', attributions), inp_f, file_id, my_key, my_env, tag="")
        else:
            print('fastqc run missing for',file_id)
    # we have a run but no qc (maybe file was reuploaded and qc was cleared)
    else:
        if not a_fastq.get('quality_metric'):
            needs_run_again += 1
            if run_fastqc == 'fq':
                print('fastqc running for', a_fastq['accession'])
                inp_f = {'input_fastq':file_id}
                run_missing_wfr(step_settings('fastqc-0-11-4-1', 'none', attributions), inp_f, file_id, my_key, my_env, tag="")
            else:
                print('fastqc run missing for',file_id)
        else:
            weird += 1

        
print("needs_run, needs_run_again, running, weird")    
print(needs_run, needs_run_again, running, weird)


Do you wanna run fastqc if missing? (fq/none)fq
6
fastqc running for 4DNFIIRK6HZQ


fastqc running for 4DNFIO2AB2XM


fastqc running for 4DNFIOHYOKHI


fastqc running for 4DNFIUF9HWHJ


fastqc running for 4DNFI3DXXQQ4


fastqc running for 4DNFI5RTSUJ2


needs_run, needs_run_again, running, weird
6 0 0 0


In [16]:
# Release fastq qc
# fastq_url = '/search/?status=released&status=released%20to%20project&type=FileFastq&limit=all'
# fastq_files = ff_utils.search_metadata(fastq_url , key=my_key)
# fastq_files = [i for i in fastq_files if i.get('quality_metric')]
print(len(fastq_files))

patch = True
need_patch = 0
need_patch_run = 0
counter = 0
for a_fastq in fastq_files:
    counter += 1
    if counter < 3400:
        continue
        
    if counter % 100 == 0:
        print()
        print(counter)
    fastq_status = a_fastq['status']
    if fastq_status in ['released', 'released to project']:
        qc_uuid = a_fastq['quality_metric']['uuid']
        qc_status = ff_utils.get_metadata(qc_uuid,key=my_key)['status']
        if qc_status != fastq_status:
            need_patch += 1
            patch_data = {"status": fastq_status}
            if patch:
                print(counter, a_fastq['accession'], a_fastq['status'], 'qc', qc_status)
                ff_utils.patch_metadata(patch_data, obj_id=qc_uuid ,key=my_key)
                #print('q', end='')
        # find the workflow that produced this qc
        query = '/search/?type=WorkflowRun&output_quality_metrics.value.uuid=' + qc_uuid
        try:
            qc_run = ff_utils.search_metadata(query, key=my_key)[0]
        except:
            print(query)
            print(ff_utils.search_metadata(query, key=my_key))
            print(a_fastq['uuid'])
            print(qc_uuid)
            continue
        assert qc_run['run_status'] == 'complete'
        if qc_run['status'] != fastq_status:
            need_patch_run += 1
            patch_data = {"status": fastq_status}
            if patch:
                print(counter, a_fastq['accession'], a_fastq['status'], 'qc_run', qc_run['status'])
                ff_utils.patch_metadata(patch_data, obj_id=qc_run['uuid'] ,key=my_key)
                #print('w', end='')

print(need_patch)
print(need_patch_run)


4874

3400

3500

3600

3700

3800

3900

4000

4100

4200

4300

4400

4500

4600

4700

4800
0
0


In [18]:
### Check if md5 runs are in sync with file status
import time
from dcicutils import s3Utils
from dcicutils import ff_utils
from functions.wfr import *
from functions.wfr_settings import *
from functions.notebook_functions import *

# get key from keypairs.json
my_env = 'data'
my_key = get_key('koray_data')


fq_runs = '/search/?run_status=complete&status=in+review+by+lab&type=WorkflowRunAwsem&workflow.title=FastQC'
fq_hits = ff_utils.search_metadata(fq_runs , key=my_key)

print(len(fq_hits))

prob = 0
for a_run in fq_hits:
    f_status = a_run['input_files'][0]['value']['status']
    if f_status != a_run['status']:
        print(f_status, a_run['status'])
        prob += 1
    
print(prob)


patch = True
need_patch = 0
need_patch_run = 0
counter = 0

123
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded in review by lab
uploaded